## AltumAge prediction example

In [17]:
#load necessary packages
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import linear_model, preprocessing

In [18]:
#load list of selected CpGsites
cpgs = np.array(pd.read_pickle('example_dependencies/CpGsites.csv'))

#load processed example data from GEO30870 data set
#ensure the methylation data has been normalized with BMIQCalibration from Horvath 2013
data = pd.read_pickle('example_dependencies/example_data.pkl')

#load standard scaler
scaler = pd.read_pickle('example_dependencies/scaler.pkl')

#load AltumAge model
AltumAge = tf.keras.models.load_model('AltumAge.h5')

In [19]:
#regardless of the Illumina platform, select *in order* the 21368 CpG sites from the list 
real_age = data.age
methylation_data = data[cpgs]

In [20]:
#scale data
methylation_data_scaled = scaler.transform(methylation_data)

In [22]:
#predict with AltumAge
pred_age_AltumAge = AltumAge.predict(methylation_data_scaled).flatten()

In [23]:
#get AltumAge evaluation metrics
mae = np.median(np.abs(real_age - pred_age_AltumAge))
mse = np.mean((real_age - pred_age_AltumAge)**2)
r = np.corrcoef(real_age, pred_age_AltumAge)[0,1]
print('The Median Absolute Error is: ' + str(mae))
print('The Mean Squared Error is: ' + str(mse))
print("Pearson's Correlation Coefficient is: " + str(r))

The Median Absolute Error is: 9.992599487304688
The Mean Squared Error is: 98.52498424905616
Pearson's Correlation Coefficient is: 0.9890003381511906


## Comparison with Horvath's 2013 model

In [7]:
#Horvath's age transformation function
def anti_transform_age(exps):
    import numpy as np
    adult_age = 20
    ages = []
    for exp in exps:
        import numpy as np
        if exp < 0:
            age = (1 + adult_age)*(np.exp(exp))-1
            ages.append(age)
        else:
            age = (1 + adult_age)*exp + adult_age
            ages.append(age)
    ages = np.array(ages)
    return ages

In [8]:
#loading model parameters for Horvath's 2013 Model
coef_data = pd.read_csv('example_dependencies/coefficients.csv')
intercept = coef_data[0:1].CoefficientTraining[0]
horvath_cpgs = np.array(coef_data.drop(0).CpGmarker)
coefs = np.array(coef_data.drop(0).CoefficientTraining)
horvath_model = linear_model.LinearRegression()
horvath_model.coef_ = coefs
horvath_model.intercept_ = intercept

In [9]:
#predict with Horvath's 2013 model
pred_ages_Horvath = anti_transform_age(horvath_model.predict(methylation_data[horvath_cpgs]))

In [10]:
#get Horvath's 2013 model evaluation metrics
mae = np.median(np.abs(real_age - pred_ages_Horvath))
mse = np.mean((real_age - pred_ages_Horvath)**2)
r = np.corrcoef(real_age, pred_ages_Horvath)[0,1]
print('The Median Absolute Error is: ' + str(mae))
print('The Mean Squared Error is: ' + str(mse))
print("Pearson's Correlation Coefficient is: " + str(r))

The Median Absolute Error is: 14.140236344179442
The Mean Squared Error is: 171.92677509854326
Pearson's Correlation Coefficient is: 0.9726306089777842
